In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [157]:
#use pandas to load the data
df = pd.read_csv("real_estate_dataset.csv")

#get number of samples and features
n_samples, n_features = df.shape

#get the names of coloumns
columns = df.columns

print("Number of samples and features ", n_samples, n_features)

print("The names of the columns are ", columns) 

# use col id 'Square_Feet',  'Garage_Size', 'Location_Score','Distance_to_Center',
X= df[['Square_Feet',  'Garage_Size', 'Location_Score','Distance_to_Center']].values

# use price as target
y = df['Price'].values

print(f"shape of X: {X.shape} \n")
print(f"shape of y: {y.dtype}\n")

#get the number of samples and feature in X
n_samples, n_features = X.shape



Number of samples and features  500 12
The names of the columns are  Index(['ID', 'Square_Feet', 'Num_Bedrooms', 'Num_Bathrooms', 'Num_Floors',
       'Year_Built', 'Has_Garden', 'Has_Pool', 'Garage_Size', 'Location_Score',
       'Distance_to_Center', 'Price'],
      dtype='object')
shape of X: (500, 4) 

shape of y: float64



In [158]:
#build a linear model to predict the price of the house from 4 features in X
#make an array fo coefs of the siz of n_features+1, initialize to 1

coefs = np.ones(n_features+1)

#predict the price for each sample in X
prediction_bydefn = X @ coefs[1:] + coefs[0]

#append a col of 1s to X
X = np.hstack([np.ones((n_samples, 1)), X])

#predict the size of each sample in X
predictions = X @ coefs

#see i f all the entries in prediction_bydefn are the same 
is_same = np.allclose(prediction_bydefn, predictions)

print(f"Are the predictions the same with X*coefs[0] and X@coefs? {is_same}")   

# calculate the error using the predictions in Y
errors = y - predictions

rel_errors = errors / y

#calculate mean of squares errors using a loop
loss_loop = 0
for i in range(n_samples):
    loss_loop += errors[i]**2   

loss_loop /= n_samples 

#calculate mean of squares errors using matrix operations
loss_matrix = np.transpose(errors) @ errors / n_samples

#compare the two methods of calculating mean squared error
is_diff = np.allclose(loss_loop, loss_matrix) 
print(f"Are the losses by direct and matrix the same? {is_diff}")

print(f"size of errors: {errors.shape} \n") 
print(f"L2 norm of errors: {np.linalg.norm(errors)} \n")
print(f"L2 norm of relative errors: {np.linalg.norm(rel_errors)} \n")


Are the predictions the same with X*coefs[0] and X@coefs? True
Are the losses by direct and matrix the same? True
size of errors: (500,) 

L2 norm of errors: 13297007.321853261 

L2 norm of relative errors: 22.35214323542266 



In [159]:
#what is my optimization problem?
#I want to find the coedds that minimize the mean squared error
#this problem is called as least squares problem
#




In [160]:
#aside
#in Nu = 

In [161]:
#objective function f(coefs) = 1/n_samples * \sum_{i=1}^{n_samples}      ?  (y_i - X_i @ coefs)^2

# what is a solution?
# a solution is a set coefs that minimizes the objective function

#how do we find a solution?
#by searchin for the coeffs at which the gradient of the objective function is zero
# OR i CAN set the gradint of objective fn to zero and solve for the coefs

# write the loss matrix in terms of data and coeffs

loss =  (y - X @ coefs).T @ (y - X @ coefs) / n_samples

#calculate the gradient of the loss function wrt coefs
gradient = -2/n_samples * X.T @ (y - X @ coefs)

#we set grad_mate=rix = 0 and solve for coefs
#X.T @ y = X.T @ X @ coefs
# X.T @ X @ coefs = X.T @ y this eqn is called normal eqn
#coeff = (X.T @ X)^-1 @  X.T @ y
coefs = np.linalg.inv(X.T @ X) @ X.T @ y

#save in a file coefs.csv
np.savetxt("coefs.csv", coefs, delimiter=",")

#predict and find errors
predictions = X @ coefs
errors = y - predictions

#print l2norm of errors
print(f"L2 norm of errors: {np.linalg.norm(errors)} \n")
#print relative errorr of errors
rel_errors = errors / y
print(f"L2 norm of relative errors: {np.linalg.norm(rel_errors)} \n")

#use all features except "price" as X
X = df.drop("Price", axis=1).values

#add bais term to X
X = np.hstack([np.ones((n_samples, 1)), X])

#calc coeffs using normal eqn
coefs_all = np.linalg.inv(X.T @ X) @ X.T @ y
#predict and find errors
predictions_all = X @ coefs_all
errors_all = y - predictions_all

#calc relative errors
rel_errors_all = errors_all / y

#print l2norm of errors
print(f"L2 norm of errors: {np.linalg.norm(errors_all)} \n")    
print(f"L2 norm of relative errors: {np.linalg.norm(rel_errors_all)} \n")
#save in a file coefs_all.csv
np.savetxt("coefs_all.csv", coefs_all, delimiter=",")


L2 norm of errors: 2240271.8037529774 

L2 norm of relative errors: 4.327097762677231 

L2 norm of errors: 423041.8687172993 

L2 norm of relative errors: 0.8097874801854511 



In [162]:
# read Matrix_Calculus wiki page

In [163]:
# most costly step inverse of X.T X
rank_XTX = np.linalg.matrix_rank(X.T @ X)
print(f"Rank of X.T @ X: {rank_XTX}")
# use QR fortorization to solve the normal eqn using matrix decomp
Q, R = np.linalg.qr(X)

print(f"shape of Q: {Q.shape}")
print(f"shape of R: {R.shape}") 

#write R TO A file R.csv
np.savetxt("R.csv", R, delimiter=",")

#R*coeffs = b
sol = Q.T @ Q
np.savetxt("sol.csv", Q, delimiter=",")

# X= QR
#X.T @ X = R.T @ Q.T @ Q @ R = R.T @ R  
#X.T @ y = R.T @ Q.T @ y
#R*coeffs = Q.T @ y

b = Q.T @ y
print(f"shape of b: {b.shape}")
print(f"shape of R: {R.shape}")

#coefs_qr = np.linglang.inv(R) @ b  
n_samples, n_features = X.shape
coefs_qr_loop = np.zeros(n_features)

#loop to solve R*coeffs = b using backsubstitution
for i in range(n_features - 1, -1, -1):
    coefs_qr_loop[i] = b[i]
    for j in range(i + 1, n_features):
        coefs_qr_loop[i] = coefs_qr_loop[i]-R[i, j] * coefs_qr_loop[j]
    coefs_qr_loop[i] =coefs_qr_loop[i]/ R[i, i]


#save coeffs_qr_loop to a file named coeffs_qr_loop.csv
np.savetxt("coeffs_qr_loop.csv", coefs_qr_loop, delimiter=",")



Rank of X.T @ X: 12
shape of Q: (500, 12)
shape of R: (12, 12)
shape of b: (12,)
shape of R: (12, 12)


In [164]:
#solve the normal eqn using SVD
# X = U  S  V.T

#eigen decomposition of sq matrix
# A = V @ D @ V.T
#A^-1 = V @ D^-1 @ V.T
#X*coeffs = y
#A= X^T  X

#normal equation: X^T X coeffs = X^T y

U, S, Vt = np.linalg.svd(X, full_matrices=False)

#Find inveerse of x in laeast squares sense
#pseudo inverse of x

In [165]:
#after class
#to complete:calclate the coeffs_svd using the SVD decomp using the pseudo inverse of X

In [166]:
#inverse matrix S
S_inv = np.diag(1/S)
coeffs_svd = Vt.T @ S_inv @ U.T @ y


#save the coeffs obtained from SVD in coeffs_svd.csv
np.savetxt("coeffs_svd.csv", coeffs_svd, delimiter=",")